<a href="https://colab.research.google.com/github/wzljerry/Hierarchical-Federated-Learning/blob/main/HFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import hashlib
import pandas as pd
import numpy as np
from sklearn import datasets
from keras.datasets import mnist
import keras
import random
import time
from random import choice
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as Ks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from keras.layers import Convolution2D
from tensorflow.keras.layers import Dropout

# to get the training data, and split the data via the number of clients
class Get_data:
  def __init__(self,n):
    self.n=n # number of clients

  def load_data(self):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    return X_train, y_train, X_test, y_test

  def flatten_data(self,X):
    data=[]
    for i in X:
      da=i.flatten()
      data.append(da)
    return data

  def one_d_to_n_d(self,X):
    data=[]
    for i in X:
      da=i.reshape(28,28)
      data.append(da)
    return data

  def non_iid(self,X,y):
    train_X=self.flatten_data(X)
    train_y=list(y)
    train_data=np.c_[train_X,train_y]
    sort_data=train_data[np.argsort(train_data[:,784])]
    train_x=sort_data[:,0:784]
    train_Y=np.array(sort_data[:,784])
    train_x_da=np.array(self.one_d_to_n_d(train_x))
    return train_x_da,train_Y

  def split_data(self, data, n): 
    size=int(len(data) / self.n)
    s_data = []
    for i in range(0, int(len(data)) + 1, size):
        c_data = data[i:i + size]
        if c_data != []:
            s_data.append(c_data)
    return s_data

  def pre_data(self):

    X_train, y_train, X_test, y_test=self.load_data()#iid

    X_train,y_train=self.non_iid(X_train,y_train) # non_iid

    X_train=self.split_data(X_train,self.n) 
    y_train=self.split_data(y_train,self.n)
    return X_train, y_train, X_test, y_test

class Model:
  
  def __init__(self):
    pass

  def global_models(self):
    model= tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(28, 28)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(10, activation='softmax')
      ])
    return model
  
  def global_model(self):
    model = Sequential()

    model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(28,28,1))) 
    model.add(Convolution2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

  def evaluate_model(self,model,test_X, test_y):
    model.compile(optimizer='sgd',
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=['accuracy'])
    score=model.evaluate(test_X, test_y)
    print("Test Loss:", score[0])
    print('Test Accuracy:', score[1])
    return score[0],score[1]

class Client:
  
  def __init__(self,lr,epoch):
    #self.n=n
    self.epoch=epoch
    self.lr = lr
    self.loss='categorical_crossentropy'
    self.metrics = ['accuracy']
    self.optimizer = SGD(lr=self.lr, 
                decay=self.lr / 2, 
                momentum=0.9
               )
  def weight_client(self,data,m,n):
    wei_client = []
    for i in range(n):
        len_data = len(data[i])
        proba = len_data / m
        wei_client.append(proba)
    return wei_client
    
  
  def scale_model_weights(self,weight,scalar,num):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)

    fac=scalar[num]

    sca=[fac for i in range(steps)]

    for i in range(steps):
      weight_final.append(sca[i]*weight[i])
        #weight_final.append(weight[i])
    return weight_final

  def training(self,X,y,global_weights):

    #fact=self.weight_client()
    model=Model().global_model()

    model.compile(optimizer=self.optimizer,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    model.set_weights(global_weights)
    model.fit(X,y,epochs=self.epoch)
    weights=model.get_weights()

    return weights

class Server:
  
  def __init__(self):
    pass
  
  def scale_model_weights(self,weight,scalar,num):
    '''function for scaling a models weights'''
    weight_final = []
    wei=[]
    steps = 8

    for j in range(num):
      fa=scalar[j]
      scar=[fa for k in range(steps)]
      for i in range(steps):
        weight_final.append(scar[i]*weight[j][i])
        #weight_final.append(weight[i])
      wei.append(weight_final)
    return wei

  def sum_scaled_weights(self,scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

  def evaluate(self,model,test_X, test_y):
    loss,acc=Model.evaluate_model(model,test_X, test_y)
    return loss,acc

class compute_estimated_weights:
  def __init__(self,weight):
    self.weight=weight

  def sum_scaled_weights(self,scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

  def scale_model_weights(self,weight,scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
      weight_final.append(scalar[i]*weight[i])
    return weight_final

  def get_difference_single_client(self):
    scalar=[-1 for i in range(100)]#len(self.weight)
    wei_d=[]
    for i in range(len(self.weight)):
      if i+1>=len(self.weight):
        pass
      else:
        sw=self.scale_model_weights(self.weight[i],scalar)
        weit_c=[self.weight[i],self.weight[i+1]]
        df=self.sum_scaled_weights(weit_c)
        wei_d.append(df)
    return wei_d
  
  def get_mean_difference_single_client(self,weight):
    scalar=[1/len(weight) for i in range(100)]#len(weight)
    wei=[]
    for i in range(len(weight)):
      sw=self.scale_model_weights(weight[i],scalar)
      wei.append(sw)
    #print(len(wei))
    mean_df=self.sum_scaled_weights(wei)
    return mean_df

  def get_estimated_weights_single_client(self):
    weit=self.get_difference_single_client()
    #wei=self.get_mean_difference_single_client(weit)
    mean_weight=self.get_mean_difference_single_client(weit)
    wei_c=[self.weight[-1],mean_weight]
    estimated_weights=self.sum_scaled_weights(wei_c)
    return estimated_weights

def reshape_his_wei(data,T,server,client):
  W=[]
  X=cut([i for i in data],int(len(data)/(T)))
  for j in range(len(X)):
    Y=cut([i for i in X[j]],int(len(X[j])/num_server)) #每个t有2个server
    Z=cut([i for i in Y[server]],int(len(Y[server])/K)) #这个server有2个k
    for h in range(len(Z)): # 进入每个k
      wei=Z[h][client]
      W.append(wei)
  return W

def reshape_egde_wei(data,num_server,t,j):
  w=[]
  X=X=cut([i for i in data],num_server)
  for i in range(len(X)):
    w.append(X[i][j])
  return w

def reshape_local_wei(data,number_client,num):
  w=[]
  X=cut([i for i in data],num_client)
  for i in range(len(X)):
    w.append(X[i][num])
  return w

def random_stragglers(t,j,i,k,t_s,j_s,i_s,k_s):
  a=False
  if t in t_s and j in j_s and i in i_s and k in k_s:
    a=True
    return a

def t_round(t,j,t_s,j_s):
  a=False
  if t in t_s and j in j_s:
    a=True
    return a

def get_order(num_server,num_client):
  w=[]
  h=[]
  X=[i for i in range(num_client*num_server)]
  X=np.array(X)
  y=X.reshape(num_server,num_client)
  y=pd.DataFrame(y)
  for i in range(num_client):
    d=list(y.iloc[:,i])
    w.append(d)
  k=list(w)
  
  for j in range(num_server):
    h.extend(k[j])
  return h

def cut(obj, sec):
  data=[obj[i:i+sec] for i in range(0,len(obj),sec)]
  return data

def shuffle_data(data):
  for i in range(len(data)):
    if (i%2==0) and i<=(len(data)/2-1):
      data[i],data[-i]=data[-i],data[i]
  return data

def factor_re(factor,num,i,t,k):
  factor[i]=(0.80**(t))*factor[i]
  #factor[i]=(1/(1+0.001*t))*factor[i]
  for j in range(len(factor)):
    if j != i:
      factor[j]=(1-factor[i])/(len(factor)-num)
  return factor

def decay_weights(data,t):
    data=(0.80**(t))*data
    return data

def HFL(num_server,K,num_client,T,lr,epoch):

  client=Client(lr,epoch)

  server=Server()
  global_model=Model().global_model()

  get_data=Get_data(num_server*num_client)
  train_X,train_y, test_X, test_y=get_data.load_data()
  X_train, y_train, X_test, y_test=get_data.pre_data()
  m=len(train_X)

  X=cut([i for i in X_train],num_client)
  y=cut([i for i in y_train],num_client)

  accuracy=[]
  losses=[]
  server_w=[]
  factor_s=[1/num_server for l in range(num_server)]
  fa=[1/num_client for h in range(10)]
  
  for t in range(T):
    global_weights=global_model.get_weights()
    server_w=[]
    for j in range(num_server):
      local_weights=global_weights
      for k in range(K):
        weit=[]
        for i in range(num_client):
          weix=client.training(X[j][i],y[j][i],local_weights)
          weix_s=client.scale_model_weights(weix, fa,i)
          weit.append(weix_s)
        local_w_k=server.sum_scaled_weights(weit)
        local_weights=local_w_k
      server_weights=client.scale_model_weights(local_w_k,factor_s,j)
      server_w.append(server_weights)
    global_weight=server.sum_scaled_weights(server_w) # fedavg
    global_model.set_weights(global_weight)
    loss,acc=Model().evaluate_model(global_model,test_X,test_y)
    losses.append(loss)
    accuracy.append(acc)
    
  return losses,accuracy

if __name__=='__main__':
  num_server=5# number of servers
  K=1 # number of local rounds
  T=50 # number of global rounds
  num_client=5# number of clients for each server
  lr=0.001 # learning rate
  epoch=1 # local iterations

  loss,acc=HFL(num_server,K,num_client,T,lr,epoch)
  loss
  print('====================================loss================================')
  print(loss)
  print('====================================acc================================')
  print(acc)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


11501568/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


313/313 [==============================] - 4s 13ms/step - loss: 3.4135 - accuracy: 0.2424
Test Loss: 3.413451671600342
Test Accuracy: 0.24240000545978546
====================================loss================================
[3.413451671600342]
====================================acc================================
[0.24240000545978546]
